# 第1回 宿題

## 課題： PyTorchを使ってMLPを実装

- データセットには、10カテゴリーの衣料品の、70,000枚のグレースケールの画像からなる[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)を用います。
    - torchvisionのdatasetsに`FashionMNIST`というクラスでデータセットが用意されているので、これを利用すると便利です。
    - `train`引数にTrue/Falseを与えるとそれぞれ訓練データとテストデータが取得できます
- 訓練データでモデルを学習し、テストデータに対する予測をcsvに出力して提出してください。
- ファイルの形式はsample_submission.csvを参考に、サンプルごとにラベルをカンマで区切って提出してください。
- ファイル名はsubmission.csvとしてください。
- テストデータに対する予測結果の精度（F1スコア）で評価します。

In [ ]:
import os
os.chdir('/root/userspace/chap1/') # 自分のファイルがあるパスにしてください

In [ ]:
! head -c 100 sample_submission.csv

以下のサンプルコードを参考にしてください。

In [ ]:
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

in_dim  = 784
hid_dim = 200
out_dim = 10
lr = 0.001
batch_size = 32
num_epochs = 1

# データセットの読み込み
train_data = torchvision.datasets.FashionMNIST(
    './data/fashion-mnist',
    transform=torchvision.transforms.ToTensor(),
    train=True,
    download=True)

test_data = torchvision.datasets.FashionMNIST(
    './data/fashion-mnist',
    transform=torchvision.transforms.ToTensor(),
    train=False,
    download=True)

# データローダの定義
train_data_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True)

test_data_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False)


In [ ]:
# モデルの定義
class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super(MLP, self).__init__()
        self.Linear1 = nn.Linear(in_dim, hid_dim)
        self.Linear2 = nn.Linear(hid_dim, out_dim)
    
    def forward(self, x):
        x = F.relu(self.Linear1(x))
        return F.log_softmax(self.Linear2(x), dim=-1)

mlp = MLP(in_dim, hid_dim, out_dim).to(device)

optimizer = optim.SGD(mlp.parameters(), lr=lr)

criterion = nn.NLLLoss()

# 学習
for i in range(num_epochs):
    for x, t in train_data_loader:        
        x = x.to(device)
        t = t.to(device)
        
        x = x.view(x.size(0), -1)
        y = mlp.forward(x)
        
        loss = criterion(y, t)
        
        loss.backward()
        
        optimizer.step()

In [ ]:
# テストデータを予測
preds = []
for x, _ in test_data_loader:
    x = x.to(device)
    x = x.view(x.size(0), -1)
    y = mlp.forward(x)
    pred = y.argmax(1).tolist()
    preds += pred
    
# 予測結果を保存
SUBMISSION_PATH = 'submission.csv'
with open(SUBMISSION_PATH, 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(preds)

In [ ]:
# admin: sample_submission.csvの作成
! head ./submission.csv > ./sample_submission.csv

In [ ]:
# admin: answer.csvの作成
! python ./admin/create_answer.py

In [ ]:
# admin: 評価
! USERSUBMISSION=submission.csv python ./admin/scoring.py